## Bi-LSTM video classification

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import seaborn as sns
from IPython.display import YouTubeVideo
import matplotlib.pyplot as plt
import plotly.plotly as py
import multiprocessing as mp # if we want to parallelize i/o

# keras imports
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model
import operator
import time 
import gc
import os

import os
from glob import glob
from tqdm import tqdm
import sys

from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
import pyspark
import numpy as np
from elephas.spark_model import SparkModel

Using TensorFlow backend.


WARNING


In [2]:
###########
#SPECIFY PARAMS FIRST
###########

# 10 class problem for now?
label_feature_size = 10

# how many frames we will use from each video?
max_frame_rgb_sequence_length = 10
frame_rgb_embedding_size = 1024

# how many audio sequences we will use from each video?
max_frame_audio_sequence_length = 10
frame_audio_embedding_size = 128

number_dense_units = 1000
number_lstm_units = 100
rate_drop_lstm = 0.2
rate_drop_dense = 0.2
activation_function='relu'
validation_split_ratio = 0.2


def create_model():
    """Create and store best model at `checkpoint` path ustilising bi-lstm layer for frame level data of videos"""
    
    # Creating 2 bi-lstm layer, one for rgb and other for audio level data
    lstm_layer_1 = Bidirectional(LSTM(number_lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
    lstm_layer_2 = Bidirectional(LSTM(number_lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
    
    # creating input layer for frame-level data
    frame_rgb_sequence_input = Input(shape=(max_frame_rgb_sequence_length, frame_rgb_embedding_size), dtype='float32')
    frame_audio_sequence_input = Input(shape=(max_frame_audio_sequence_length, frame_audio_embedding_size), dtype='float32')
    frame_x1 = lstm_layer_1(frame_rgb_sequence_input)
    frame_x2 = lstm_layer_2(frame_audio_sequence_input)
    
    #creating input layer for video-level data 
    vid_shape=(1024,)
    video_rgb_input = Input(shape=vid_shape)
    video_rgb_dense = Dense(int(number_dense_units/2), activation=activation_function, input_shape=vid_shape)(video_rgb_input)
    
    aud_shape=(128,)
    video_audio_input = Input(shape=aud_shape)
    video_audio_dense = Dense(int(number_dense_units/2), activation=activation_function,input_shape = aud_shape)(video_audio_input)
    
    # merging frame-level bi-lstm output and later passed to dense layer by applying batch-normalisation and dropout
    merged_frame = concatenate([frame_x1, frame_x2])
    merged_frame = BatchNormalization()(merged_frame)
    merged_frame = Dropout(rate_drop_dense)(merged_frame)
    merged_frame_dense = Dense(int(number_dense_units/2), activation=activation_function)(merged_frame)
    
    # merging video-level dense layer output
    merged_video = concatenate([video_rgb_dense, video_audio_dense])
    merged_video = BatchNormalization()(video_rgb_dense)
    merged_video = Dropout(rate_drop_dense)(merged_video)
    merged_video_dense = Dense(int(number_dense_units/2), activation=activation_function)(merged_video)

    
    # merging frame-level and video-level dense layer output
    merged = concatenate([merged_frame_dense, merged_video_dense])
    merged = BatchNormalization()(merged)
    merged = Dropout(rate_drop_dense)(merged)
     
    merged = Dense(number_dense_units, activation=activation_function)(merged)
    merged = BatchNormalization()(merged)
    merged = Dropout(rate_drop_dense)(merged)
    preds = Dense(label_feature_size, activation='sigmoid')(merged)
    
    model = Model(inputs=[frame_rgb_sequence_input, frame_audio_sequence_input, video_rgb_input, video_audio_input], outputs=preds)

    print(model.summary())
    
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['acc'])

    return model

In [3]:
def extract_video_files(video_files_path):    
    '''
    Extraction of Youtube tfrecords video file features.
    
    Args: path to video files (note: developed with assumption of storing on s3 bucket and assessing with glob)
    
    Assumes each video in the tfrecord has following features:
    'id' : bytes_list
    'labels' : int64_list
    'mean_rgb': float_list
    'mean_audio': float_list
    
    returns:
    numpy arrays of video ids, video multi-labels, mean rgb and mean audio
    '''
    
    vid_ids = []
    labels = []
    mean_rgb = []
    mean_audio = []

    for file in tqdm(glob(video_files_path)):
        for example in tf.python_io.tf_record_iterator(file):
            tf_example = tf.train.Example.FromString(example)

            vid_ids.append(tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
            labels.append(tf_example.features.feature['labels'].int64_list.value)
            mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
            mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)
            
    assert len(vid_ids) == len(labels),"The number of IDs does not match the number of labeled videos."
    return vid_ids, labels, mean_rgb, mean_audio


In [15]:
def extract_frame_level_features_per_tf_record(frame_file_path,maximum_iter = False,stop_at_iter = 10):
    '''
    Extraction of Youtube tfrecords frame file features.
    
    Args: 
    path to each tf_record (note: developed with assumption of storing on s3 bucket and assessing with glob)
    
    maximum_iter - flag- if True, will limit number of videos extracted from each TF record
    stop_at_iter - number of videos to extract
    num_tf_records - number of records to extract - WARNING!!! this is VERY slow, if bigger than 1
    
    Assumes each video in the tfrecord has following features:
    'id' : bytes_list
    'labels' : int64_list
    'audio': float arr, each frame 128
    'rgb', float arr, each frame 1024
    
    returns:
    numpy arrays of frame ids, frame multi-labels, frame audio, frame rgb
    '''
    frame_ids = []
    frame_labels = []
    feat_rgb = []
    feat_audio = []
    # ATTENTION: only use one TF record for debugging.
    print(f'There is {sum(1 for _ in tf.python_io.tf_record_iterator(frame_file_path))} videos in this TF record.')
    iter_ = 0
    for example in tf.python_io.tf_record_iterator(frame_file_path):
        if maximum_iter and iter_==stop_at_iter:
            break
        tf_example = tf.train.Example.FromString(example)

        frame_ids.append(tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
        frame_labels.append(tf_example.features.feature['labels'].int64_list.value)

        tf_seq_example = tf.train.SequenceExample.FromString(example)
        n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)

        rgb_frame = []
        audio_frame = []

        # iterate through frames
        sys.stdout.flush()
        for i in range(n_frames):
            sess = tf.InteractiveSession()
            sys.stdout.write('\r'+'iterating video: ' + str(iter_)+ ' ,frames: ' + str(i)+'/'+str(n_frames))
            sys.stdout.flush()
            rgb_frame.append(tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval())
            audio_frame.append(tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval())

            tf.reset_default_graph()
            sess.close()
        feat_rgb.append(rgb_frame)
        feat_audio.append(audio_frame)
        iter_+=1

    return frame_ids, frame_labels, feat_rgb, feat_audio

In [5]:
def create_train_dev_dataset(video_rgb, video_audio, vid_ids, frame_rgb, frame_audio, frame_labels, frame_ids):
    """
    Method to created training and validation data. 
    We need to make sure we only use video IDs for which we have frames.
    This is handled below.
    
    """
    # we have to have the same video of for both video and frame-level features
    video_rgb_matching = []
    video_audio_matching = []
    
    for idx in frame_ids: # for each ID available on frame level, find matching video-level features
        for i, idx_vid in enumerate(vid_ids): # scan through video-level ids
            if idx == idx_vid: 
                video_rgb_matching.append(video_rgb[i])
                video_audio_matching.append(video_audio[i])
                
                
    shuffle_indices = np.random.permutation(np.arange(len(frame_labels)))
        
    video_rgb_shuffled = np.array(video_rgb_matching)[shuffle_indices]
    video_audio_shuffled = np.array(video_audio_matching)[shuffle_indices]
    frame_rgb_shuffled = np.array(frame_rgb)[shuffle_indices]
    frame_audio_shuffled = np.array(frame_audio)[shuffle_indices]
    labels_shuffled = np.array(frame_labels)[shuffle_indices]

    dev_idx = max(1, int(len(labels_shuffled) * validation_split_ratio))
    
    # delete orig vars to clear some cache
    del video_rgb
    del video_audio
    del frame_rgb
    del frame_audio
    gc.collect()
    
    train_video_rgb, val_video_rgb = video_rgb_shuffled[:-dev_idx], video_rgb_shuffled[-dev_idx:]
    train_video_audio, val_video_audio = video_audio_shuffled[:-dev_idx], video_audio_shuffled[-dev_idx:]
    
    train_frame_rgb, val_frame_rgb = frame_rgb_shuffled[:-dev_idx], frame_rgb_shuffled[-dev_idx:]
    train_frame_audio, val_frame_audio = frame_audio_shuffled[:-dev_idx], frame_audio_shuffled[-dev_idx:]
    
    train_labels, val_labels = labels_shuffled[:-dev_idx], labels_shuffled[-dev_idx:]
    
    del video_rgb_shuffled, video_audio_shuffled, frame_rgb_shuffled, frame_audio_shuffled, labels_shuffled
    gc.collect()
    
    return (train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio, train_labels, val_video_rgb, val_video_audio, 
            val_frame_rgb, val_frame_audio, val_labels)

In [7]:
# transform into final input in the model
def one_hot_y(raw_labels,label_size=20):
    '''
    Helper function to transform labels into one-hot TOP 20
    Uses np.unique(return_counts=True) as implicit sorter (first K labels are the most frequent)
    '''
    all_labels = []
    for i in list(raw_labels):
        for j in list(i):
            all_labels.append(j)

    results = np.unique(all_labels,return_counts=True)
    labels_vocab,counts = results

    labels = labels_vocab[:label_size-1] #last columns will be 1 if none of those labels found in a video
    output = []
    for set_of_labels in raw_labels:
        
        # preallocate numpy arr for each set of labels
        sequence = np.zeros(label_size)
        # loop through all the labels in one video and flip them to 1s
        for this_label in set_of_labels:
            designation = np.where(labels==this_label)
            for des in designation:
                sequence[des]=1
        # done with one training points
        if sequence.sum()==0:
            sequence[-1]=1
        output.append(sequence)
    return output


In [8]:
def transform_data_for_lstm(video_rgb,video_audio, frame_rgb, frame_audio,
                            labels,label_feature_size=10,max_frame_rgb_sequence_length = 10,\
                            max_frame_audio_sequence_length = 10):
    frames = []
    # need to transfrom to numpy (num_videos x max_frame_rgb_sequence_length x 1024)
    #print(len(frame_rgb))
    
    for frame in frame_rgb: 
        # stack the frames in each video, only allowed number of first frams
        #print(np.vstack(frame).shape)
        frames.append(np.vstack(frame)[:max_frame_rgb_sequence_length,:])
    #print(len(frames))

    frames = np.reshape(np.array(frames),(len(frame_rgb),max_frame_rgb_sequence_length,1024))

    #print(frames.shape)
    
    frames_audio = []
    # need to transfrom to numpy (num_videos x max_frame_audio_sequence_length x 128)
    for frame in frame_audio:
        # stack the frames in each video, only allowed number of first frams
        #print(np.vstack(frame).shape)
        frames_audio.append(np.vstack(frame)[:max_frame_audio_sequence_length,:])

    frames_audio = np.reshape(np.array(frames_audio),(len(frame_audio),max_frame_audio_sequence_length,128))
    #print(frames_audio.shape)
    
    # deal with videos
    
    video_rgb = np.vstack(video_rgb)
    video_audio = np.vstack(video_audio)
    
    
    # labels - need to one-hot encode TOP - K label
    labels = one_hot_y(labels,label_feature_size)
    labels = np.vstack(labels)
    return frames,frames_audio, video_rgb,video_audio, labels

# run on SPARK

In [9]:
def create_pandas(frame_rgb, frame_audio, video_rgb, video_audio, labels):
    '''
    Spark unfortunately does not work with numpy arrays - so we need to convert to traditional python types.
    '''
    return pd.DataFrame.from_dict({'frame_rgb':[[[float(k) for k in j] for j in i] for i in frame_rgb],\
                             'frame_audio':[[[float(k) for k in j] for j in i] for i in frame_audio],\
                             'mean_rgb':[[float(j) for j in i] for i in list(video_rgb)],\
                            'mean_audio':[[float(j) for j in i] for i in list(video_audio)],\
                            'labels':[[float(j) for j in i] for i in list(labels)]})

In [10]:
conf = SparkConf().setAppName('Youtube-8M') \
                  .set("spark.jars",
                       "ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.11-1.10.0.jar")
sc = SparkContext(conf = conf)
spark = pyspark.sql.SparkSession(sc)

In [11]:
# We just load all of the video data in memory since it is fairly small and manageable.
train_vid_ids, train_labels, train_mean_rgb, train_mean_audio \
    = extract_video_files("mys3bucket/yt8pm_100th_shard/v2/video/train*")

  0%|          | 0/41 [00:00<?, ?it/s]

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


100%|██████████| 41/41 [00:04<00:00,  9.51it/s]


In [12]:
# the fun starts here, pull frame data per tf-record
train_frame_shards = glob('mys3bucket/yt8pm_100th_shard/v2/frame/train*')

In [16]:
# iterate tf records one by one and append to spark dataframe

FIRST_RECORD = True # flag whether to append to Spark SQL or create a new one
NUM_RECORDS_TO_LOAD = 1

for tf_record in tqdm(train_frame_shards[:NUM_RECORDS_TO_LOAD]):
    # pull frames in memory
    train_frame_ids, train_frame_labels,train_frame_rgb,train_frame_audio \
        = extract_frame_level_features_per_tf_record(tf_record,maximum_iter=True,\
                               stop_at_iter=10) # just pull 10 videos from each tf record for debugging
    # first transformation
    train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio, \
    train_labels, val_video_rgb, val_video_audio, val_frame_rgb, val_frame_audio, val_labels \
                = create_train_dev_dataset(train_mean_rgb, train_mean_audio, train_vid_ids, train_frame_rgb, \
                                            train_frame_audio, train_frame_labels, train_frame_ids )    
    
    # final transformation for LSTM
    train_frame_rgb, train_frame_audio, train_video_rgb, train_video_audio, train_labels = \
            transform_data_for_lstm(train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio,train_labels)
    
    val_frame_rgb, val_frame_audio, val_video_rgb, val_video_audio, val_labels = \
            transform_data_for_lstm( val_video_rgb, val_video_audio,val_frame_rgb, val_frame_audio, val_labels)
    
    #### BELOW WE ONLY USE THE TRAINING DATA AND NO VALIDATION DATA FOR SIMPLICITY
    df = create_pandas(train_frame_rgb, train_frame_audio, \
                   train_video_rgb, train_video_audio, train_labels)
    # create spark data frame
    if FIRST_RECORD:
        df_spark = spark.createDataFrame(df)
        FIRST_RECORD = False
    else:
        df_spark_new = spark.createDataFrame(df)
        df_spark = df_spark.union(df_spark_new)
        
    

  0%|          | 0/2 [00:00<?, ?it/s]

There is 1015 videos in this TF record.
iterating video: 9 ,frames: 122/123

 50%|█████     | 1/2 [00:33<00:33, 33.85s/it]

There is 1029 videos in this TF record.
iterating video: 9 ,frames: 127/128

100%|██████████| 2/2 [01:03<00:00, 32.53s/it]


In [ ]:
## DYLAN - YOUR JOB IS TO FIGURE OUT HOW TO RUN THE ELEPHAS TRAINING NOW

In [17]:
df_spark.count()

16

In [18]:
df_spark.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|         frame_audio|           frame_rgb|              labels|          mean_audio|            mean_rgb|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[WrappedArray(101...|[WrappedArray(53....|[1.0, 0.0, 0.0, 0...|[-1.0705330371856...|[-1.1001880168914...|
|[WrappedArray(173...|[WrappedArray(170...|[0.0, 1.0, 0.0, 0...|[1.12153804302215...|[-0.0249195266515...|
|[WrappedArray(173...|[WrappedArray(0.0...|[0.0, 1.0, 0.0, 0...|[-0.2137299776077...|[-0.3027234375476...|
|[WrappedArray(155...|[WrappedArray(105...|[1.0, 0.0, 0.0, 0...|[-0.9770614504814...|[-0.5987871885299...|
|[WrappedArray(183...|[WrappedArray(209...|[0.0, 0.0, 0.0, 0...|[0.93089854717254...|[0.51462107896804...|
|[WrappedArray(192...|[WrappedArray(127...|[0.0, 0.0, 1.0, 1...|[1.11850774288177...|[-0.3764120936393...|
|[WrappedArray(103...|[WrappedArray(9

In [19]:
train_rdd = df_spark.rdd

In [20]:
# change the order of vars to fit the order in the model!
# train_frame_rgb, train_frame_audio, train_video_rgb, train_video_audio], train_labels
train_rdd = train_rdd.map(lambda x: (np.array(x[1]), np.array(x[0]),np.array(x[4]),np.array(x[3]),np.array(x[2])))

In [21]:
keras_model= create_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 1024)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 10, 128)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 200)          900000      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidi

In [22]:
spark_model = SparkModel(keras_model, frequency='batch', mode='synchronous')

In [23]:
# HOW TO MAKE SURE THIS WORKS???
history = spark_model.fit(train_rdd, epochs=10, batch_size=2, verbose=0)

>>> Fit model


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 5.0 failed 1 times, most recent failure: Lost task 3.0 in stage 5.0 (TID 20, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/ubuntu/anaconda3/envs/te